## CHEME 5660 The Tiger Problem as a Markov Decision Problem

### Introduction
Fill me in

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Linear-Tiger-MDP`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Linear-Tiger-MDP/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Linear-Tiger-MDP/Manifest.toml`


In [2]:
using Distributions
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1664


In [66]:
include("CHEME-5660-Tiger-MDP-CodeLib.jl");

In [4]:
# setup some global constants -
α = 0.75; # probability of moving the direction we are expect

#### States and actions

In [5]:
# setup the states and actions -
safety = 1;
tiger = 50;

states = range(safety,stop=tiger, step=1) |> collect;
actions = [1,2,3]; # a₁ = move left, a₂ = move right
γ = 0.95;

#### Rewards

In [6]:
# setup the rewards -
R = Array{Float64,2}(undef,length(states), length(actions));
fill!(R,0.0) # fill R w/zeros

# set the rewards for the ends -
R[safety + 1,1] = 10;
R[tiger-1, 2] = -100;
R[1:length(states), 3] .= -1;

#### Transitions

In [7]:
# Setup the transitions
T = Array{Float64,3}(undef, length(states), length(states), length(actions));
fill!(T,0.0);

# We need to put values into the transition array (these are probabilities, so eah row much sum to 1)
T[safety, 1, 1:length(actions)] .= 1.0; # if we are in state 1, we stay in state 1 ∀a ∈ 𝒜
T[tiger, tiger, 1:length(actions)] .= 1.0; # if we are in state 5, we stay in state 5 

##### Left, Right and Listen Actions

In [16]:
# left actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,1] = α;
    T[s,s+1,1] = (1-α);
end

# right actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,2] = (1-α);
    T[s,s+1,2] = α; 
end

# listen action (we don't move to a new state)
for s ∈ 2:(tiger-1)
    T[s,s,3] = 1.0;
end

### Build the MDP problem object and estimate the utility $U^{\pi}(s)$ 

In [75]:
m = build(MDP; 𝒮 = states, 𝒜 = actions, T = T, R = R, γ = γ);

In [11]:
# build a always right policy -
always_move_right(s) = 2;
always_move_left(s) = 1;

In [76]:
U = iterative_policy_evaluation(m, always_move_right, 20*length(states));

### Estimate the Q-Array

In [78]:
Q_array = Q(m, U)[2:end-1,:]

48×3 Matrix{Float64}:
    9.66039    -1.01884    -1.9679
   -1.12191    -1.42995    -2.35845
   -1.46141    -1.66733    -2.58396
   -1.67713    -1.86346    -2.77029
   -1.86673    -2.05961    -2.95663
   -2.06093    -2.26953    -3.15605
   -2.27029    -2.49877    -3.37383
   -2.49939    -2.75053    -3.613
   -2.75116    -3.02747    -3.8761
   -3.02814    -3.33224    -4.16563
   -3.33297    -3.66767    -4.48429
   -3.66848    -4.03686    -4.83502
   -4.03775    -4.44321    -5.22105
    ⋮                    
  -40.3476    -44.3993    -43.1793
  -44.409     -48.8685    -47.4251
  -48.8793    -53.7876    -52.0982
  -53.7995    -59.2019    -57.2418
  -59.2149    -65.1612    -62.9031
  -65.1755    -71.7203    -69.1343
  -71.7361    -78.9397    -75.9927
  -78.9571    -86.8858    -83.5415
  -86.9049    -95.6317    -91.8501
  -95.6528   -105.258    -100.995
 -105.281    -115.853    -111.061
  -82.5455   -127.515    -122.139

In [74]:
policy = π(Q_array)

50-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1